In [295]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd

In [296]:
# open the url
html = urlopen("https://www.yourinvestmentpropertymag.com.au/top-suburbs/vic/")
bsobj = BeautifulSoup(html, "lxml")

In [297]:
#find suburb's href
n=0
suburb_href = []
for l in bsobj.find_all('a', href=re.compile("^(/top-suburbs/)((?!:).)*$")):
    if 'href' in l.attrs:
        n = n +1
        suburb_href.append(l.attrs['href'])
    if n == 9:
        break
suburb_href.pop(0)
state =[]
for states in suburb_href:
    state.append(states.split('/')[-2])
state

['nsw', 'wa', 'act', 'vic', 'sa', 'tas', 'qld', 'nt']

In [298]:
labels =['state',
 'suburb',
 'postcode',
 'url',
 'Median_price_house',
 'Median_price_unit',
 'Quarterly_growth_house',
 'Quarterly_growth_unit'
 'twelve_month_growth_house',
 'twelve_month_growth_unit',
 'Average_annual_growth_house',
 'Average_annual_growth_unit',
 'Weekly_median_rent_house',
 'Weekly_median_rent_unit',
 'Gross_rental_yield_house',
 'Gross_rental_yield_unit',
 'Number_of_Sales_(12m)_house',
 'Number_of_Sales_(12m)_unit',
 'Avg._Days_on_Market_(12m)_house',
 'Avg._Days_on_Market_(12m)_unit',
 'Total_population_2011',
 'Total_population_2016',
 'Population_change_(5y)_2011',
 'Population_change_(5y)_2016',
 'Median_household_income_(p/w)_2011',
 'Median_household_income_(p/w)_2016',
 'Household_income_change_(5y)_2011',
 'Household_income_change_(5y)_2016',
 'Median_age_of_persons_2011',
 'Median_age_of_persons_2016',
 'DESC_house',
 'DESC_unit']

In [299]:
df = pd.DataFrame(columns=labels)

In [310]:
home_link = 'https://www.yourinvestmentpropertymag.com.au'
i= 0 

for n in range(len(suburb_href)):
    state_url = home_link + suburb_href[n]
    response = requests.get(state_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for href in soup.find_all('a', class_="visible-item"):

        if len(state[n]) == 3:
            suburb_url = state_url + href['href'][17:]

            df.loc[i,'state'] = state[n]
            df.loc[i,'url'] = suburb_url
            df.loc[i,'postcode'] = href['href'][17:].split('-')[0]
            df.loc[i,'suburb'] = href['href'][17:].split('-',1)[1:][0].replace('-',' ')
            suburb_response = requests.get(suburb_url)
            suburb_soup = BeautifulSoup(suburb_response.content, 'html.parser')
            c = 4
            x = 19
            for t in suburb_soup.find('div', {'class' : 'key-market-data'}).find_all('span'):
                data = t.text.replace("$","")
                data = data.replace(",","")
                data = data.replace("+","")
                if data == '–':
                    data = 'Null'
                df.loc[i,labels[c]] = data
                c= c+1
            for t in suburb_soup.find('div', {'class' : 'key-demographics'}).find_all('span'):
                data = t.text.replace("$","")
                data = data.replace(",","")
                data = data.replace("+","")
                if data == '–':
                    data = 'Null'
                df.loc[i,labels[x]] = data
                x= x+1
        else:
            suburb_url = state_url + href['href'][16:]
            df.loc[i,'state'] = state[n]
            df.loc[i,'url'] = suburb_url
            df.loc[i,'postcode'] = href['href'][16:].split('-')[0]
            df.loc[i,'suburb'] = href['href'][16:].split('-',1)[1:][0].replace('-',' ')
            suburb_response = requests.get(suburb_url)
            suburb_soup = BeautifulSoup(suburb_response.content, 'html.parser')
            c = 4
            x = 19
            for t in suburb_soup.find('div', {'class' : 'key-market-data'}).find_all('span'):
                data = t.text.replace("$","")
                data = data.replace(",","")
                data = data.replace("+","")
                if data == '–':
                    data = 'Null'
                df.loc[i,labels[c]] = data
                c= c+1
            for t in suburb_soup.find('div', {'class' : 'key-demographics'}).find_all('span'):
                data = t.text.replace("$","")
                data = data.replace(",","")
                data = data.replace("+","")
                if data == '–':
                    data = 'Null'
                df.loc[i,labels[x]] = data
                x= x+1


        i = i +1 



In [313]:
for i in range(len(df)):
    suburb_url = df['url'][i]
    suburb_response = requests.get(suburb_url)
    suburb_desc_soup = BeautifulSoup(suburb_response.content, 'html.parser')
    DESC_house = suburb_desc_soup.find('div', {'aria-labelledby' :'pills-house-report-tab'}).text.replace('\n','')
    df.loc[i,'DESC_house'] = DESC_house
    DESC_unit = suburb_desc_soup.find('div', {'id' :'pills-unit-report'}).text.replace('\n','')
    df.loc[i,'DESC_unit'] = DESC_unit

In [314]:
df

,state,suburb,postcode,url,Median_price_house,Median_price_unit,Quarterly_growth_house,Quarterly_growth_unittwelve_month_growth_house,twelve_month_growth_unit,Average_annual_growth_house,...,Population_change_(5y)_2011,Population_change_(5y)_2016,Median_household_income_(p/w)_2011,Median_household_income_(p/w)_2016,Household_income_change_(5y)_2011,Household_income_change_(5y)_2016,Median_age_of_persons_2011,Median_age_of_persons_2016,DESC_house,DESC_unit
0,nsw,aarons pass,2850,https://www.yourinvestmentpropertymag.com.au/t...,Null,Null,Null,Null,Null,Null,...,35.67,-94.80,531,825,-38.47,55.37,45,53,Aarons Pass has 1 house available for sale. Th...,The size of Aarons Pass is approximately 82.8 ...
1,nsw,abbotsbury,2176,https://www.yourinvestmentpropertymag.com.au/t...,1510000,Null,1.34,Null,16.15,Null,...,0.12,1.90,1956,2146,22.56,9.71,35,38,Abbotsbury has 40 houses available for sale. T...,There were 0 unit sales in the past 12 months...
2,nsw,abbotsford,2046,https://www.yourinvestmentpropertymag.com.au/t...,3045000,1235000,-6.09,-6.30,2.35,-5.54,...,3.38,5.00,1765,1966,19.42,11.39,42,44,Abbotsford has 31 houses available for sale. T...,Abbotsford has 59 units available for sale. Th...
3,nsw,abercrombie,2795,https://www.yourinvestmentpropertymag.com.au/t...,822500,Null,4.78,Null,25.57,Null,...,-40.63,7.50,1783,2098,20.64,17.67,33,36,Abercrombie has 22 houses available for sale. ...,There were 0 unit sales in the past 12 months...
4,nsw,abercrombie river,2795,https://www.yourinvestmentpropertymag.com.au/t...,Null,Null,Null,Null,Null,Null,...,56.14,-96.60,831,737,4.79,-11.31,45,43,Abercrombie River has 1 house available for sa...,There were 0 unit sales in the past 12 months...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,nt,howard springs,0835,https://www.yourinvestmentpropertymag.com.au/t...,697500,Null,4.42,Null,7.39,Null,...,16.46,62.90,1927,2222,26.44,15.31,41,41,Howard Springs has 42 houses available for sal...,There were 0 unit sales in the past 12 months...
796,nt,hugh,0872,https://www.yourinvestmentpropertymag.com.au/t...,Null,Null,Null,Null,Null,Null,...,-98.84,-11.00,950,1028,-16.89,8.21,32,33,"The size of Hugh is approximately 9,536 square...","The size of Hugh is approximately 9,536 square..."
797,nt,hughes,0822,https://www.yourinvestmentpropertymag.com.au/t...,Null,Null,Null,Null,Null,Null,...,20.27,-88.10,1599,Null,46.03,Null,37,52,Hughes has 1 house available for sale. There w...,There were 0 unit sales in the past 12 months...
798,nt,humpty doo,0836,https://www.yourinvestmentpropertymag.com.au/t...,625000,Null,0.81,Null,4.17,Null,...,31.39,5.80,1740,2256,37.22,29.66,36,36,Humpty Doo has 61 houses available for sale. T...,Humpty Doo has 6 units available for sale. The...
